# Mansa loans

## EDA

We start by performing an EDA of the datasets.

In [3]:
import pandas as pd
from src.data_tools import get_data

In [4]:
# read the csv files
df_accounts = get_data("data/accounts.csv")
df_transactions = get_data("data/transactions.csv")

In [5]:
df_accounts.head()

,id,balance,update_date
0,0,13.63,2021-07-02
1,1,12.91,2021-07-02
2,2,19.84,2021-07-02
3,3,130.00,2021-07-02
4,4,2806.75,2021-07-02


In [6]:
df_transactions.head()

,account_id,date,amount
0,0,2020-10-16,200.0
1,0,2020-10-16,-192.0
2,0,2020-10-16,200.0
3,0,2020-10-16,-24.0
4,0,2020-10-16,-50.0


#### Checking for null values

In [7]:
df_accounts.isna().any()

id             False
balance        False
update_date    False
dtype: bool

In [8]:
df_transactions.isna().any()

account_id    False
date          False
amount        False
dtype: bool

There is no null value in any of the two datasets.


#### Checking for duplicates

In [9]:
df_accounts.duplicated().any()

False

In [10]:
df_transactions.duplicated().any()

True

There are duplicates in the transactions. Lets look at some of them

In [11]:
n_duplicated = df_transactions.duplicated().sum()
dup_percent = int(n_duplicated / df_transactions.shape[0]*100)
print(f'there are {n_duplicated} duplicates in transactions.csv, representing {dup_percent}% of the data.')

there are 28661 duplicates in transactions.csv, representing 5% of the data.


In [12]:
df_dup = df_transactions[df_transactions.duplicated()]
df_dup.head()

,account_id,date,amount
2,0,2020-10-16,200.0
11,0,2020-10-17,0.0
14,0,2020-10-19,0.0
23,0,2020-10-21,0.0
45,0,2020-10-29,-5.5


In [13]:
df_transactions[
    (df_transactions.account_id == 0) 
    & (df_transactions.date == '2020-10-16') 
    &(df_transactions.amount == 200.0)]

,account_id,date,amount
0,0,2020-10-16,200.0
2,0,2020-10-16,200.0


It is difficult, without further information if the above transactions are legitimate siilar transactions or duplicates coming from an error when the data were processed. We decide to keep these duplicates. It would be interesting to enrich the transactions data with a transaction id. We could be certain if a similar transaction on a given account and a given date is a duplicate or legitimate.

#### Account history distribution

Let's check the history distribution of the different accounts in our data.

In [14]:
# we change the date column from str to datetime
df_transactions['date'] = pd.to_datetime(df_transactions['date'])
group = df_transactions.groupby('account_id')['date']

In [15]:
history_df = group.agg(["min", "max"])
history = history_df['max'] - history_df['min']
history.head()

account_id
0   257 days
1   231 days
2   100 days
3   176 days
4   293 days
dtype: timedelta64[ns]

In [16]:
trigger = pd.Timedelta(180, "d")
n_accounts = (history > trigger).sum()
frac_long_account = int(n_accounts / len(history) * 100)
print(f'{n_accounts} accounts, which represent {frac_long_account}% of all the accounts, have an history of more than 6 months')

823 accounts, which represent 65% of all the accounts, have an history of more than 6 months


We'll focus on the accounts with more than 6 months history. For that purpose we use get_df_with_history.

In [17]:
from src.data_tools import get_df_with_history
df_accounts, df_transactions = get_df_with_history(df_accounts, df_transactions)

In [28]:
n_transac = len(set(df_transactions['account_id']))
print(f'We are left with {len(df_accounts)} accounts in df_accounts and {n_transac} in df_transactions which is as expected.')

We are left with 823 accounts in df_accounts and 823 in df_transactions which is as expected.


SyntaxError: invalid syntax (<ipython-input-25-8e464d705049>, line 1)